<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1euQsumOq2on57AQOXo3kuiWHBuqlxmKt?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

# 🚀**Building_AgentLite_AI_Agents**  

AgentLite is an open-source framework for building task-oriented AI agents using LLMs.  
It enables seamless development of both single-agent and multi-agent architectures.  
Designed for research, it simplifies agent orchestration and interaction.  

### 🔹 **Key Features:**  
 ✅ Lightweight & easy-to-use framework  
 ✅ Supports multi-agent collaboration  
 ✅ Customizable for various AI applications

###**Setup and Installation**

In [ ]:
!pip install agentlite-llm chess

In [ ]:
!pip install --upgrade langchain langchain-openai

### 🔑 **Setup OpenAI API Key**








In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

### 🤖 **Creating a Hello World Agent with AgentLite**








In [ ]:
from agentlite.agents import BaseAgent
from agentlite.llm.agent_llms import get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.commons import TaskPackage
from agentlite.llm.agent_llms import LangchainChatModel


class HelloWorldAgent(BaseAgent):
    def __init__(self, llm):
        super().__init__(name="HelloWorld", role="Responds with Hello World!", llm=llm)

    def respond(self, task, **kwargs):
        return "Hello World!"

llm_config = LLMConfig({"llm_name": "gpt-3.5-turbo", "temperature": 0.0, "api_key":OPENAI_API_KEY})
llm = LangchainChatModel(llm_config)

hello_agent = HelloWorldAgent(llm=llm)

### 🎯 **Executing Task with Hello World Agent**








In [ ]:
task = TaskPackage(instruction="Any instruction here")
response = hello_agent(task)
print(response)

/usr/local/lib/python3.11/dist-packages/agentlite/llm/agent_llms.py:91: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm_chain.run(prompt)


Agent HelloWorld receives the following TaskPackage:
[
	Task ID: d0addc8c-e2a2-4ab9-9401-fa435b0388c6
	Instruction: Any instruction here
]
====HelloWorld starts execution on TaskPackage d0addc8c-e2a2-4ab9-9401-fa435b0388c6====
Agent HelloWorld takes 0-step Action:
{
	name: Think
	params: {'response': 'I need to respond with Hello World! as per my role.'}
}
Observation: OK
Agent HelloWorld takes 1-step Action:
{
	name: Finish
	params: {'response': 'Hello World!'}
}
Observation: Hello World!
=========HelloWorld finish execution. TaskPackage[ID:d0addc8c-e2a2-4ab9-9401-fa435b0388c6] status:
[
	completion: completed
	answer: Hello World!
]
Hello World!


### ♟️ **Integrating Chess with AgentLite**








In [ ]:
from typing import List
from agentlite.actions import BaseAction
from agentlite.agents import ABCAgent, BaseAgent
from agentlite.llm.agent_llms import BaseLLM, get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging import AgentLogger
import chess
import chess.svg
from IPython.display import display, HTML
from agentlite.commons import AgentAct, TaskPackage
from agentlite.actions import ThinkAct, FinishAct
from agentlite.actions.InnerActions import INNER_ACT_KEY

### 🔥 **Implementing Chess Move Action in AgentLite**








In [ ]:
BOARD = chess.Board()

class BoardMove(BaseAction):
    def __init__(self) -> None:
        action_name = "BoardMove"
        action_desc = "Using this action to move a step on chess board"
        params_doc = {"query": "Standard Universal Chess Interface Move"}
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )

    def __call__(self, query):
        try:
            BOARD.push_uci(query)
        except ValueError as e:
            return f"Error: {e}. The movement is illegal. I need to find a legal move with LegalMoves action."

        else:
            m = chess.Move.from_uci(query)
            svg_code = chess.svg.board(BOARD, arrows = [(m.from_square, m.to_square)], fill = {m.from_square: "gray"}, size = 200)

            display(HTML(svg_code))
        return f"Moved on board, move is {query}. I should finish my turn."


### 👀 **Viewing Chess Board State**








In [ ]:
class BoardView(BaseAction):
    def __init__(self) -> None:
        action_name = "BoardView"
        action_desc = "Get the Forsyth-Edwards Notation of current board status"
        params_doc = {"key": "True"}
        super().__init__(
            action_name = action_name,
            action_desc = action_desc,
            params_doc = params_doc
        )

    def __call__(self, key, **kwargs):
        return BOARD.fen()


### ✅ **Fetching Legal Moves in Chess with AgentLite**








In [ ]:
class LegalMoves(BaseAction):
    def __init__(self) -> None:
        action_name = "LegalMoves"
        action_desc = "Get the legal moves of current board status"
        params_doc = {"key": "True"}
        super().__init__(
            action_name = action_name,
            action_desc = action_desc,
            params_doc = params_doc
        )

    def __call__(self, key):
        legal_moves = list(BOARD.legal_moves)
        legal_moves = [m.uci() for m in legal_moves]
        return ', '.join(legal_moves)

agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)


### 🎭 **Creating a Chess Player Agent in AgentLite**








In [ ]:
class ChessPlayerAgent(BaseAgent):
    def __init__(
        self,
        llm: BaseLLM,
        actions: List[BaseAction] = [BoardMove(), BoardView(), LegalMoves()],
        color = "",
        **kwargs
    ):
        if color not in ["white", "black"]:
            raise ValueError(f"color must be either white or black, but got {color}")
        opponent_color = "black" if color == "white" else "white"

        name = f"{color}_player"
        opponent_name = f"{opponent_color}_player"

        role = f"""Your name is {name} and you are a chess player. You are playing against {opponent_name}. You are playing as {color}. """
        constraint = """You communicate your move using universal chess interface language.
        Step1: View the board with BoardView action.
        Step2: Get the legal moves with LegalMoves action.
        Step3: Make a legal move using BoardMove action.
        """
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            logger=agent_logger,
            constraint=constraint,
            **kwargs
        )


### ⚔️ **Initializing Chess Player Agents**








In [ ]:
player_white = ChessPlayerAgent(llm = llm, color = "white")
player_black = ChessPlayerAgent(llm = llm, color = "black")

### 🔄 **Simulating a Chess Game Round with AgentLite**








In [ ]:

exp_task = "It is your turn."
exp_task_pack = TaskPackage(instruction=exp_task)
act_1 = AgentAct(
    name=BoardView().action_name,
    params={"key": "True"
    },
)
obs_1 = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

act_2 = AgentAct(
    name=LegalMoves().action_name,
    params={"key": "True",
        },
)
obs_2 = "g1h3, g1f3, b1c3, b1a3, h2h3, g2g3, f2f3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4, g2g4, f2f4, e2e4, d2d4, c2c4, b2b4, a2a4"

act_3 = AgentAct(
    name=BoardMove().action_name,
    params={"query": "g1h3",
        },
)
obs_3 = """Moved on board, move is g1h3. It is opponent's turn next."""

act_4 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "Chess Moved. It's my opponent's turn."})
obs_4 = "Task Completed."
exp_act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3), (act_4, obs_4)]

player_white.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)
player_black.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)
num_round = 1

for i in range(num_round):
    exp_task = "It is your turn"
    exp_task_pack = TaskPackage(instruction=exp_task)
    player_white(exp_task_pack)

    exp_task = "It is your turn"
    exp_task_pack = TaskPackage(instruction=exp_task)
    player_black(exp_task_pack)

Agent white_player receives the following TaskPackage:
[
	Task ID: d0addc8c-e2a2-4ab9-9401-fa435b0388c6
	Instruction: It is your turn
]
====white_player starts execution on TaskPackage d0addc8c-e2a2-4ab9-9401-fa435b0388c6====
Agent white_player takes 0-step Action:
{
	name: BoardView
	params: {'key': 'True'}
}
Observation: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Agent white_player takes 1-step Action:
{
	name: LegalMoves
	params: {'key': 'True'}
}
Observation: g1h3, g1f3, b1c3, b1a3, h2h3, g2g3, f2f3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4, g2g4, f2f4, e2e4, d2d4[TLDR]
Agent white_player takes 2-step Action:
{
	name: BoardMove
	params: {'query': 'g1f3'}
}


Observation: Moved on board, move is g1f3. I should finish my turn.
Agent white_player takes 3-step Action:
{
	name: Finish
	params: {'response': "Chess Moved. It's black_player's turn."}
}
Observation: Chess Moved. It's black_player's turn.
=========white_player finish execution. TaskPackage[ID:d0addc8c-e2a2-4ab9-9401-fa435b0388c6] status:
[
	completion: completed
	answer: Chess Moved. It's black_player's turn.
]
Agent black_player receives the following TaskPackage:
[
	Task ID: d0addc8c-e2a2-4ab9-9401-fa435b0388c6
	Instruction: It is your turn
]
====black_player starts execution on TaskPackage d0addc8c-e2a2-4ab9-9401-fa435b0388c6====
Agent black_player takes 0-step Action:
{
	name: BoardView
	params: {'key': 'True'}
}
Observation: rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1
Agent black_player takes 1-step Action:
{
	name: LegalMoves
	params: {'key': 'True'}
}
Observation: g8h6, g8f6, b8c6, b8a6, h7h6, g7g6, f7f6, e7e6, d7d6, c7c6, b7b6, a7a6, h7h5, g7g5, f7f5, e7e5, d7

Observation: Moved on board, move is g8f6. I should finish my turn.
Agent black_player takes 3-step Action:
{
	name: Finish
	params: {'response': "Chess Moved. It's white_player's turn."}
}
Observation: Chess Moved. It's white_player's turn.
=========black_player finish execution. TaskPackage[ID:d0addc8c-e2a2-4ab9-9401-fa435b0388c6] status:
[
	completion: completed
	answer: Chess Moved. It's white_player's turn.
]
